# Analysis Code for 4-Point Correlator

## Preparation


In [1]:
%pip install numpy scipy iminuit matplotlib scienceplots
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import scienceplots

Note: you may need to restart the kernel to use updated packages.


Lattice size:


In [2]:
xyz_size = 32
t_size = 64
t_half = int(t_size / 2)
xyz_cube = int(xyz_size**3)

Directory and file names:


In [3]:
droot = "/Volumes/X6/data/ccbar"

gfix = ["gfix_C", "gfix_L"]
gauge = ["Coulomb", "Landau"]
chan = ["ps", "v"]
binID = ["BIN" + f"{i:02}" for i in range(1, 19)]

### Time reversal

Calculate $C_{\text{ave}}(n_t) = [C(n_t) + C(N_t - n_t)]/2$


In [4]:
for ig in range(2):
    ofpath = f"{droot}/{gfix[ig]}/4pt/trev"
    os.system(f"rm -rf {ofpath}")  # clear data
    os.makedirs(ofpath, exist_ok=True)  # make directory for output files
    for ich in range(2):
        for it in range(t_half):
            t1 = f"{it:+03}"
            t2 = f"{(t_size - it)%t_size:+03}"
            for ibin in range(18):
                ifname_t1 = f"{droot}/{gfix[ig]}/4pt/binned/4pt.{chan[ich]}.{t1}.{gfix[ig]}.{binID[ibin]}"
                ifname_t2 = f"{droot}/{gfix[ig]}/4pt/binned/4pt.{chan[ich]}.{t2}.{gfix[ig]}.{binID[ibin]}"
                ofname = f"{droot}/{gfix[ig]}/4pt/trev/{chan[ich]}.{t1}.{gfix[ig]}.{binID[ibin]}"
                assert os.path.exists(ifname_t1)
                assert os.path.exists(ifname_t2)

                os.system(
                    f"bin/mean -l {xyz_cube} -o {ofname} {ifname_t1} {ifname_t2}"
                )

            print(f"{gauge[ig]} gauge, {chan[ich]} chan: got [C({t1}) + C({t2})]/2")

Coulomb gauge, ps chan: got [C(+00) + C(+00)]/2
Coulomb gauge, ps chan: got [C(+01) + C(+63)]/2
Coulomb gauge, ps chan: got [C(+02) + C(+62)]/2
Coulomb gauge, ps chan: got [C(+03) + C(+61)]/2
Coulomb gauge, ps chan: got [C(+04) + C(+60)]/2
Coulomb gauge, ps chan: got [C(+05) + C(+59)]/2
Coulomb gauge, ps chan: got [C(+06) + C(+58)]/2
Coulomb gauge, ps chan: got [C(+07) + C(+57)]/2
Coulomb gauge, ps chan: got [C(+08) + C(+56)]/2
Coulomb gauge, ps chan: got [C(+09) + C(+55)]/2
Coulomb gauge, ps chan: got [C(+10) + C(+54)]/2
Coulomb gauge, ps chan: got [C(+11) + C(+53)]/2
Coulomb gauge, ps chan: got [C(+12) + C(+52)]/2
Coulomb gauge, ps chan: got [C(+13) + C(+51)]/2
Coulomb gauge, ps chan: got [C(+14) + C(+50)]/2
Coulomb gauge, ps chan: got [C(+15) + C(+49)]/2
Coulomb gauge, ps chan: got [C(+16) + C(+48)]/2
Coulomb gauge, ps chan: got [C(+17) + C(+47)]/2
Coulomb gauge, ps chan: got [C(+18) + C(+46)]/2
Coulomb gauge, ps chan: got [C(+19) + C(+45)]/2
Coulomb gauge, ps chan: got [C(+20) + C(

We simply denote $C_{\text{ave}}(n_t)$ as $C(n_t)$ from below


### $A_1^+$ projection

$$
\phi(r)=\frac{1}{48}\sum_{g\in O_h}\phi(g^{-1}(\mathbf{x}-\mathbf{y}))
$$

In [5]:
for ig in range(2):
    ifpath = f"{droot}/{gfix[ig]}/4pt/trev"
    ofpath = f"{droot}/{gfix[ig]}/4pt/a1plus"
    os.system(f"rm -rf {ofpath}")  # clear data
    os.makedirs(ofpath, exist_ok=True)  # make directory for output files

    os.system(f"bin/a1plus -n {xyz_size} -d {ofpath} {ifpath}/*")

    print(f"{gauge[ig]} gauge: A1+ projection done")

Coulomb gauge: A1+ projection done
Landau gauge: A1+ projection done


### Jackknife resampling

In [6]:
for ig in range(2):
    ifpath = f"{droot}/{gfix[ig]}/4pt/a1plus"
    ofpath = f"{droot}/{gfix[ig]}/4pt/jksamp"
    os.system(f"rm -rf {ofpath}")  # clear data
    os.makedirs(ofpath, exist_ok=True)  # make directory for output files

    os.system(f"bin/jre -l {xyz_cube} -d {ofpath} {ifpath}/*")

    print(f"{gauge[ig]} gauge: jk resampling done")

Coulomb gauge: jk resampling done
Landau gauge: jk resampling done


## Correlator


## Pre-potential


In [7]:
for ig in range(2):
    ifpath = f"{droot}/{gfix[ig]}/4pt/jksamp"
    ofpath = f"{droot}/{gfix[ig]}/4pt/prev"
    os.system(f"rm -rf {ofpath}")  # clear data
    os.makedirs(ofpath, exist_ok=True)  # make directory for output files

    os.system(f"bin/prev -n {xyz_size} -d {ofpath} {ifpath}/*")

    print(f"{gauge[ig]} gauge: got [▽^2 C(r,t)]/C(r,t)")

Coulomb gauge: got [▽^2 C(r,t)]/C(r,t)
Landau gauge: got [▽^2 C(r,t)]/C(r,t)


## Kawanai-Sasaki function

## Central potential of $c\bar{c}$

### Spin-independent potential

$$
V_0(r)=\frac{1}{4m_c}\Bigl[3\frac{\nabla^2 \phi_V(r)}{\phi_V(r)}+\frac{\nabla^2 \phi_{PS}(r)}{\phi_{PS}(r)}\Bigr]+\frac{1}{4}(3M_V+M_{PS})-2m_c
$$

In [8]:
for ig in range(2):
    ofpath = f"{droot}/{gfix[ig]}/4pt/potential"
    os.system(f"rm -rf {ofpath}")  # clear data
    os.makedirs(ofpath, exist_ok=True)  # make directory for output files
    mc = [0.88817, 0.69968]
    for it in range(t_half):
        t = f"{it:+03}"
        for ibin in range(18):
            prev_PS = f"{droot}/{gfix[ig]}/4pt/prev/ps.{t}.{gfix[ig]}.{binID[ibin]}"
            prev_V = f"{droot}/{gfix[ig]}/4pt/prev/v.{t}.{gfix[ig]}.{binID[ibin]}"
            ofname_v0 = f"{droot}/{gfix[ig]}/4pt/potential/v0.{t}.{gfix[ig]}.{binID[ibin]}"
            ofname_vs = f"{droot}/{gfix[ig]}/4pt/potential/vs.{t}.{gfix[ig]}.{binID[ibin]}"
            assert os.path.exists(prev_PS)
            assert os.path.exists(prev_V)

            os.system(
                f"bin/v-ti -l {xyz_cube} -mps 1.3665 -mv 1.4148 -mc {mc[ig]}  -ov0 {ofname_v0} -ovs {ofname_vs} {prev_V} {prev_PS}"
            )

    print(f"{gauge[ig]} gauge: got potentials")

Coulomb gauge: got potentials
Landau gauge: got potentials
